1. What is a perceptron?
    A perceptron is the building block of a neural network that resembles that of a biological neuron.
2. What are the different types of perceptron? Briefly describe each

In [2]:
import boto3
import pandas as pd; pd.set_option('display.max_column', 100)
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import recall_score

## define bucket in which you are trying to reach
s3 = boto3.resource('s3')
bucket_name = 'daltondencklau-data445-bucket'
bucket = s3.Bucket(bucket_name)

## define csv file to read in the bucket
file_key= 'framingham.csv'

## syntax to allow us to read the file
bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## reading the data file
heart = pd.read_csv(file_content_stream)
heart.head()

/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [3]:
## droping missing values
heart = heart.dropna()

In [3]:
## defining target and input variables
X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

In [4]:
## creating loop to build models

## creating a loop to store results
md1_results = list()
md2_results = list()
md2_results = list()
md3_results = list()

for i in range (0,100):
    
    ## splitting the data into 80% training and 20% testing
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)
    
    ## Scaling the data (NN are sensitve to input variables)
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    
    ## defining the modesl
    md1 = tf.keras.models.Sequential([
        
        ## defining the model with 1 layer, 4 neurons
        tf.keras.layers.Dense(4, input_dim = 5, activation = 'tanh'),
        
        ## defining how many layers (# of neurons, activation for output)
        tf.keras.layers.Dense(4, activation = 'softmax')
        
    ])
    
    ## defining how model will be estimated
    md1.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics= ['accuracy']) 
    md1.fit(X_train, Y_train, epochs = 100, batch_size = 500, validation_data = (X_test, Y_test))
    
    ## ## making predictions on test data set (comparing predictions and actual values)
    nn_pred1 = md1.evaluate(X_test, Y_test)
    
    ## creating a 15% cutoff value
    nn_label1 = np.where(nn_pred1 < 0.15, 0, 1)
    
    ## calculating recall score
    md1_recall = recall_score(Y_test, nn_label1)

Epoch 1/100


ValueError: in user code:

    /home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:801 train_function  *
        return step_function(self, iterator)
    /home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:791 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/tensorflow/python/